In [ ]:
import pandas as pd
import os
import time

from typing import Dict, List
from Bio import Entrez
from pathlib import Path

In [ ]:
# setup
def setup_result_directory(result_dir:str)->str:
    '''
    Creates a directory by using a specified string as argument.

    :param
        result_dir: directory path as string
    :return:
        result_dir_path: Path based on the specified result_dir string argument.
    '''
    try:
        
        result_dir_path = result_dir
        if os.path.isdir(result_dir_path) == False:
            print("[*] Trying to create specified directory: {}.".format(result_dir))
            try:
                os.mkdir(result_dir)
                print("\t[+] DONE")
            except Exception as e:
                raise Exception("[-] Not able to create directory with exception: {}".format(e))
        else:
            print("[*] Directory already exist.")
        return result_dir_path
    except Exception as e:
        raise Exception("[-] ERROR creating result directory with exception: {}".format(e))

def setup_entrez(email:str) -> None:
    '''
    Setup for the biopython Entrez.email field.

    :param
        email: String variable describing the user email.
    :return:
    '''
    Entrez.email = email

In [ ]:
# pipeline step 1 - query NCBI based on a search string
def fetch_bioprojects(query:str, max_returns=100)->List[int]:
    '''
    Function to perform an Entrez search query on the bioproject database.

    :param
        query: search string
    :param
        max_returns: maximal number of bioprojects to return
    :return:
        bioproject_search_results["IdList"]: List[int] of bioproject identifier that can be used for fetching detailed information
    '''
    try:
        bioproject_search_handle = Entrez.esearch(db="bioproject", term=query, retmax=max_returns)
        bioproject_search_results = Entrez.read(bioproject_search_handle)
        bioproject_search_handle.close()
        print("[*] Info: Found {} BioProject's associated to Exaiptasia microbiome projects.".format(bioproject_search_results["Count"]))
        return bioproject_search_results["IdList"]
    except Exception as e:
        raise Exception("[-] ERROR during fetching bioproject data with exception: {}".format(e))

# pipeline step 2 - get detailed information based on Entrez bioproject identifier
def fetch_detailed_bioproject_infos(bioproject_identifier:List[int])->List[Dict]:
    '''
    Function for extracting detailed information of bioproject entries.

    :param
        bioproject_identifier: List[int]: list of bioproject identifier
    :return:
        bioproject_docsum: List[Dict]: detailed information of the requested bioprojects.
    '''
    try:
        to_fetch_ids = ",".join(bioproject_identifier)
        bioproject_docsum_handle = Entrez.efetch(db="bioproject", id=to_fetch_ids, rettype="docsum", retmode="xml")
        bioproject_docsum = Entrez.read(bioproject_docsum_handle)["DocumentSummarySet"]["DocumentSummary"]
        bioproject_docsum_handle.close()
        print("[*] Info: Found detailed information for: {} associated BioProject's".format(len(bioproject_docsum)))
        return bioproject_docsum
    except Exception as e:
        raise Exception("[-] ERROR during fetching bioproject detailed informarion: {}".format(e))

# pipeline step 3 - writing results into CSV file
def write_bioproject_result_file(bioprojects_resultfile:str,project_document_summary:List[Dict])->None:
    '''
    Function for extracting document summary information fields. The information are written into a CSV file.

    :param
        bioprojects_resultfile: str: path to result file
    :param
        project_document_summary: List[Dict]: Entrez document summary
    :return
        None but writes a CSV file
    '''
    try:
        with open(bioprojects_resultfile,"w") as biofile:
            header = "ProjectId\tProjectAcc\tProjectDate\tProjectTitle\tProjectDescription\tOrganismName\tOrganismStrain\n"
            biofile.write(header)
            for project in project_document_summary:
                project_id = project["Project_Id"]
                project_acc = project["Project_Acc"]
                project_date = project["Registration_Date"]
                project_title = project["Project_Title"]
                project_description = project["Project_Description"]
                project_organism = project["Organism_Name"]
                project_strain = project["Organism_Strain"]
                entry = "{}\t{}\t{}\t{}\t{}\t{}\t{}\n".format(project_id,
                                                        project_acc,
                                                        project_date,
                                                        project_title,
                                                        project_description,
                                                        project_organism,
                                                        project_strain)
                biofile.write(entry)
        print("[*] INFO: done writing result file with {} entries.".format(len(project_document_summary)))
    except Exception as e:
        raise Exception("[-] ERROR during BioProject result file writing with exception: {}".format(e))

# pipeline step 4 - fetch biosample information for SRR retrieval
def get_biosample_from_bioprojects(bioproject_dataframe:pd.DataFrame)->Dict:
    '''
    This function reads a bioproject pandas dataframe, extrcats the bioproject identifier and links those to the corresponding
    biosamples using the elink function of Entrez. The fetched biosample identifier are stored in a List, which is used as a value
    for a dictionary, containing the corresponding bioproject identifiers as keys.

    :param
        bioproject_dataframe: pd.DataFrame: pandas dataframe of bioprojects, must contain a ProjectId column containing valid bioproject identifier
    :return
        bioprojects_to_biosamples: Dict: dictionary with bioproject identifier as keys and biosample identifiers as List values
    '''
    try:
        bioprojects_to_biosamples = {}
        print("[*] Linking bioproject identifier to biosample database.")
        if "ProjectId" not in list(bioproject_dataframe.columns):
            raise Exception("[-] ERROR there is no column called ProjectId in the bioproject dataframe.")
        bioproject_identifier = bioproject_dataframe.ProjectId.apply(lambda identifier: str(identifier)).to_list()
        attempts = 3
        for project_id in bioproject_identifier:
            # try querying NCBI three times
            for attempt in range(1, attempts + 1):
                try:
                    biosample_links = Entrez.elink(dbfrom="bioproject",db="biosample", id=project_id)
                    biosample_link_results = Entrez.read(biosample_links)
                    biosample_links.close()
                    break # exit attempt loop if function succeeded
                except Exception as e:
                    print("\t[-] ERROR during elink procedure with exception: {}".format(e))
                    if attempt == attempts:
                        raise Exception(e)
                    else:
                        print("\t[*] WARNING: trying elink procedure again after 1 second of sleep ...")
                        time.sleep(1)

            biosample_ids = []
            for link_set in biosample_link_results:
                for link in link_set["LinkSetDb"][0]["Link"]:
                    link_id = str(link["Id"])
                    if link_id not in biosample_ids:
                        biosample_ids.append(link_id)
            bioprojects_to_biosamples[project_id] = biosample_ids
            print("\t[*] Found {} entries for project: {}.".format(len(biosample_ids), project_id))
        print("[+] DONE fetching information for biosamples")
        return bioprojects_to_biosamples
    except Exception as e:
        raise Exception("[-] ERROR linking bioprojects to biosamples with exception: {}".format(e))

# pipeline step 5 - fetch SRR data from bioprojects
def get_sra_from_bioprojects(bioproject_dataframe:pd.DataFrame)->Dict:
    '''
    This function reads a bioproject pandas dataframe, extrcats the bioproject identifier and links those to the corresponding
    sra entries using the elink function of Entrez. The fetched sra identifier are stored in a List, which is used as a value
    for a dictionary, containing the corresponding bioproject identifiers as keys.

    :param
        bioproject_dataframe: pd.DataFrame: pandas dataframe of bioprojects, must contain a ProjectId column containing valid bioproject identifier
    :return
        bioprojects_to_sra: Dict: dictionary with bioproject identifier as keys and sra identifiers as List values   
    '''
    try:
        bioprojects_to_sra = {}
        print("[*] Linking bioproject identifier to srr database")
        if "ProjectId" not in list(bioproject_dataframe.columns):
            raise Exception("[-] ERROR there is no column called ProjectId in the bioproject dataframe.")
        bioproject_identifier = bioproject_dataframe.ProjectId.apply(lambda identifier: str(identifier)).to_list()
        attempts = 3
        for project_id in bioproject_identifier:
            # try querying NCBI three times
            for attempt in range(1, attempts + 1):
                try:
                    srr_links = Entrez.elink(dbfrom="bioproject",db="sra", id=project_id)
                    srr_link_results = Entrez.read(srr_links)
                    srr_links.close()
                    break # exit attempt loop if function succeeded
                except Exception as e:
                    print("\t[-] ERROR during elink procedure with exception: {}".format(e))
                    if attempt == attempts:
                        raise Exception(e)
                    else:
                        print("\t[*] WARNING: trying elink procedure again after 1 second of sleep ...")
                        time.sleep(1)
                        
            link_list = []
            for link_set in srr_link_results:
                if "LinkSetDb" in link_set.keys():
                    links = link_set["LinkSetDb"]
                    for sset in links:
                        if "DbTo" in sset.keys():
                            if sset["DbTo"] == "sra":
                                link_list_sets = sset["Link"]
        
                                for lset in link_list_sets:
                                    new_sra_id = str(lset["Id"])
                                    if new_sra_id not in link_list:
                                        link_list.append(new_sra_id)
            if len(link_list) == 0:
                print("[*] WARNING: no hits for: {}".format(project_id))
            else:
        
                bioprojects_to_sra[project_id] = link_list
            print("\t[*] INFO: found {} entries for project: {}".format(len(bioprojects_to_sra[project_id]), project_id))
        
        print("[+] DONE fetching information for srr")
        return bioprojects_to_sra
    except Exception as e:
        raise Exception("[-] ERROR linking bioprojects to srr with exception: {}".format(e))

In [ ]:
# helper functions
def store_elink_mapping_data(sample_dictionary:Dict,output_filename:str, header:tuple, separator="\t")->pd.DataFrame:
    '''
    This function takes a dictionary containing Lists as values. It iterates over the keys and list-values and
    outputs a CSV file containing the key - value information.

    :param
        sample_dictionary: Dict: dictionary containing a key and a list as value (e.g. bioproject identifier and corresponding sra links)
    :param
        output_filename: str: name and path of the output CSV file
    :param
        header: tuple(str, str): tuple of length two containing the key and value information as strings
    :returns
        pd.DataFrame
    '''
    try:
        print("[*] Writing output file to disk ...")
        if len(header) != 2:
            raise Exception("[-] ERROR as the header has not a size of two, choose something like: BioProjectID, SRA")
        else:
            with open(output_filename, "w") as mapfile:
                mapfile.write(header[0]+separator+header[1]+"\n")
                for key in sample_dictionary.keys():
                    for val in sample_dictionary[key]:
                        mapfile.write(key + separator + val + "\n")
        return pd.read_csv(output_filename, sep=separator)
    except Exception as e:
        raise Exception("[-] ERROR writing key - value file with exception: {}".format(e))

def read_csv_file(document_summary_csv:str, separator=",")->pd.DataFrame:
    '''
    Function for reading a csv table with pandas.
    :param
        document_summary_csv: str: Path to bioproject result file.
        separator: str: Specified separator for the CSV file.
    :return:
        pd.DataFrame
    '''
    try:
        return pd.read_csv(document_summary_csv, sep=separator)
    except Exception as e:
        raise Exception("[-] ERROR opening pandas table with exception: {}".format(e))

In [ ]:
query = '("Exaiptasia diaphana"[Organism] AND microbiome)'
bioproject_detailed_info_path = "../results/exaiptasia_microbiome_studies.csv"

setup_entrez("lukas.becker@hhu.de")
setup_result_directory("../results/")
# step 1 get bioproject identifier
bioproject_target_ids = fetch_bioprojects(query=query)
# step 2 get detailed information
bioproject_document_summary = fetch_detailed_bioproject_infos(bioproject_target_ids)
# step 3 write result file
write_bioproject_result_file(bioproject_detailed_info_path,bioproject_document_summary)
bioproject_dataframe = read_csv_file(bioproject_detailed_info_path, separator="\t")

In [ ]:
# result view
bioproject_dataframe.head()

In [ ]:
# step 4 fetch biosample information - if you get a NCBI C++ error repeat executing this function
bioprojects_to_biosamples_dictionary = get_biosample_from_bioprojects(bioproject_dataframe)

In [ ]:
# step 5 fetch srr information
bioprojects_to_sra_dictionary = get_sra_from_bioprojects(bioproject_dataframe)

In [ ]:
# step 6 - write results to disc
bioproject_to_biosample_path = "../results/bioproject_to_biosample.table"
bioproject_to_sra_path = "../results/bioproject_to_sra.table"
bioproject_to_biosample_df = store_elink_mapping_data(bioprojects_to_biosamples_dictionary, 
                                                      output_filename=bioproject_to_biosample_path,
                                                      header=("BioProjectID","BioSampleID"))
bioproject_to_sra_df = store_elink_mapping_data(bioprojects_to_sra_dictionary, 
                                                output_filename=bioproject_to_sra_path,
                                                header=("BioProjectID","SraID"))